# MGL 869 - Hive - Groupe 8 - Lab 

---


## Packages

In [1]:
import pandas as pd
import os
import git
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

## Global variables

In [2]:
HIVE_CSV_path = 'ApacheHiveCSV'
HIVE_CSV_separator = '^'
GIT_HIVE_path = 'repo'
GIT_url = 'git@github.com:apache/hive.git'

## OS environment

In [3]:
os.makedirs('output', exist_ok=True)
current_project_path = os.getcwd()

## Git environment

In [4]:
# Parent directory
parent_directory = os.path.dirname(current_project_path)
# Repository path
repo_path = current_project_path + '/' + GIT_HIVE_path

## Extract data from Hive

### Function to create dataframe from multiple sources

In [5]:
def createHiveDataFrame():
    """
    Create a dataframe from multiple CSV files
    :return: A dataframe that contains all the data from the directory HIVE_CSV_path
    """
    # Get all the CSV files in the directory
    csv_files = [os.path.join(HIVE_CSV_path, file) for file in os.listdir(HIVE_CSV_path) if file.endswith('.csv')]
    
    # Create a dataframe for each CSV file
    dataframes = [pd.read_csv(file, sep="^") for file in csv_files]
    
    dataframes_filtered = []
    for df in dataframes:
        
        # Get the columns that contain the patch versions
        fix_version_columns = [col for col in df.columns if col.startswith('Fix Version/s')]
        
        # Get the columns that contain the affects versions
        affects_version_columns = [col for col in df.columns if col.startswith('Affects Version/s')]
        
        # Combine the versions into a single column
        df['Fix Versions Combined'] = df[fix_version_columns].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
        df['Affects Versions Combined'] = df[affects_version_columns].apply(lambda x: ', '.join(x.dropna().astype(str)),
                                                                            axis=1)
        # Delete the columns that are not needed
        df = df.drop(fix_version_columns, axis=1)
        df = df.drop(affects_version_columns, axis=1)
    
        # Keep only the columns that are needed
        keep: list = ['Issue key', 'Status', 'Resolution', 'Created', 'Fix Versions Combined', 'Affects Versions Combined']
        df = df.loc[:, keep]
    
        dataframes_filtered.append(df)
    return dataframes_filtered

### Create dataframe

In [6]:
df_merged = pd.concat(createHiveDataFrame(), ignore_index=True, sort=False)

### Collect bug ids

In [7]:
# Create a set of all the ids
ids = set(df_merged[df_merged.columns[df_merged.columns.str.contains('Issue key')]].values.flatten())

## Git research
### Clone if repo isn't already clone

In [8]:
# Check if repo exists
if not os.path.exists(repo_path) or not os.path.exists(os.path.join(repo_path, '.git')):
    repo = git.Repo.clone_from(GIT_url, repo_path)
else:
    repo = git.Repo(repo_path)

### Regex to find bug names in commits

In [9]:
pattern = re.compile(r'HIVE-(\d{3,5})')

### Function to find files modified for a list of commit

In [10]:
# Function to process a batch of commits
def process_commits(commits):
    local_repo = git.Repo(repo_path) # Load the repository in memory of the current thread
    tuple_key_file_commit = []
    for commit_id in commits:
        for match in commits[commit_id]:
            hive_key = f'HIVE-{match}'
            if hive_key in ids:
                for file in local_repo.commit(commit_id).stats.files:
                    tuple_key_file_commit.append((hive_key, file, commit_id))
    return tuple_key_file_commit

### Prepare research in batch bug research in commits

In [11]:
# Get the number of threads of the CPU
num_threads = os.cpu_count() 
# Size of the chunk
chunk_size = len(list(repo.iter_commits())) // num_threads

# Get all commits and files
all_commits = [{} for _ in range(num_threads)]

for i, commit in enumerate(repo.iter_commits()):
    matches = pattern.findall(commit.message)
    if matches:
        all_commits[i // chunk_size][commit.hexsha] = matches


In [12]:
# Initialize the list to store all couples
all_couples = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_commits, chunk) for chunk in all_commits]
    for future in as_completed(futures):
        couples = future.result()
        all_couples.extend(couples)

print(f"{len(all_couples)} couples found.")

4526 couples found.


### Create the dataframe

In [13]:
# Convert the list of tuples into a DataFrame
df_files = pd.DataFrame(all_couples, columns=['Issue Key', 'File', 'Commit'])

### Keep only Java and C++ files

In [14]:
df_filtered = df_files[df_files['File'].str.endswith(('.java', '.cpp', '.c', '.h'))]

### Create tag dictionary to get only release versions

In [15]:
# Get all tags
tags = repo.tags
versions = {}

for tag in tags:
    # Get the commit of the tag
    commit = tag.commit
    commit_date = commit.committed_datetime.strftime('%Y-%m-%d')
    versions[tag.name] = commit_date

# Afficher les versions et leurs dates
# print(versions)

### Filter tags to keep only release versions

In [16]:
filtered_versions = {tag: date for tag, date in versions.items() if tag.startswith('release-') and '-rc' not in tag}
cleaned_versions = {tag.replace('release-', ''): date for tag, date in filtered_versions.items()}
# Display the cleaned versions
# print(cleaned_versions)

In [17]:
# Convertir le dictionnaire des versions en DataFrame
df_versions = pd.DataFrame(list(cleaned_versions.items()), columns=['Version', 'Release Date'])

In [18]:
# Sauvegarder dans un fichier CSV
# df_versions.to_csv("versions.csv", index=False)

In [19]:
commits_for_versions = {}
for index, row in df_versions.iterrows():
    version = row['Version']
    date = row['Release Date']
    
    # Obtenir le dernier commit avant la date de version
    commit = repo.git.log('--before', date, '-n', '1', '--pretty=format:%H')
    commits_for_versions[version] = commit

print("Derniers commits pour chaque version :", commits_for_versions)

Derniers commits pour chaque version : {'0.10.0': 'a786579fa0bb3245adea9c19d0da5fbbe7930f64', '0.11.0': '2c87e89b0305f42337070104a0d179c8910696f8', '0.12.0': 'd8ef271429d588413e42f30ecb3efae99ef25819', '0.13.0': '12b8bc55c4b2ff8bc9662e31fe88a209655f6f84', '0.13.1': '3fcc32002d8ce3f010cac0423f3cce69a90a4b09', '0.14.0': '5dc2367785e01dbe703790931a339d88e777c7a4', '0.3.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.4.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.4.1': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.5.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.6.0': 'd8701ea32ab24e4c19d583ba9b40f4789703245b', '0.7.0': 'eb72421c75c56c2fa62527c326f7835e128caca8', '0.7.1': '4f5b4e22cc19e85de8f3bfda29551954bd69a8ee', '0.8.0': '6e21fe1d209be0569b33f9c3f79d0d2fbdc50507', '0.8.1': '3555cf32326280f2b1c4391ae7fb911763941df5', '0.9.0': '09e9a233748c6b25b5485e6a8b3886bc7bede89d', '1.0.0': '7be040c5acc842f771e40f8fe0b6dca37c14a729', '1.0.1': '31383348c1bd32b7991a20b793a362cca19987e9', 